In [3]:
# coefficient matching algorithm for constrained terms
# Ax = b
# xi = 1/1024*qi1 + 1/2048*qi2 - 1/1024*qi3 - 1/2048qi4
# Ay = 2048b

import numpy as np
import random, math
import copy, sys

A = np.array([[ 16,  8,  2], [ 24, -1024,  48], [ 96, -32,  16]])
orgl_b = np.array([0.02050781, 0.55859375, 0.1640625 ])
b = 2048*orgl_b

[Dimension, ny] = A.shape
qubits = 2

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = 2*qubits*i + l
            po2 = 2*qubits*i + l + qubits
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2
 
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = 2*qubits*i + l1
                po2 = 2*qubits*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 2*qubits*i + l1 + qubits
                po4 = 2*qubits*i + l2 + qubits
                QM[po3][po4] = QM[po3][po4] + qcef
 
### Second quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = 2*qubits*i + l1
                    po2 = 2*qubits*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef
                    po3 = 2*qubits*i + l1 + qubits
                    po4 = 2*qubits*j + l2 + qubits
                    QM[po3][po4] = QM[po3][po4] + qcef
                    po5 = 2*qubits*i + l1
                    po6 = 2*qubits*j + l2 + qubits
                    QM[po5][po6] = QM[po5][po6] - qcef
                    po7 = 2*qubits*i + l1 + qubits
                    po8 = 2*qubits*j + l2
                    QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range(2*qubits*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",2*qubits*Dimension,"','q",2*qubits*Dimension,"'):",format(QM[2*qubits*Dimension-1][2*qubits*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range(2*qubits*Dimension-1):
    for j in range(i+1,2*qubits*Dimension):
        if QM[i][j] != 0:
            qdrt1 = i + 1
            qdrt2 = j + 1
            if i == 2*qubits*Dimension-2 and j == 2*qubits*Dimension-1:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
            else:
                print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")      

# Matrix Q is
[[-1.107200e+05  4.019200e+04  0.000000e+00  0.000000e+00 -5.504000e+04
  -1.100800e+05  5.504000e+04  1.100800e+05  5.440000e+03  1.088000e+04
  -5.440000e+03 -1.088000e+04]
 [ 0.000000e+00 -2.013440e+05  0.000000e+00  0.000000e+00 -1.100800e+05
  -2.201600e+05  1.100800e+05  2.201600e+05  1.088000e+04  2.176000e+04
  -1.088000e+04 -2.176000e+04]
 [ 0.000000e+00  0.000000e+00  1.308160e+05  4.019200e+04  5.504000e+04
   1.100800e+05 -5.504000e+04 -1.100800e+05 -5.440000e+03 -1.088000e+04
   5.440000e+03  1.088000e+04]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  2.817280e+05  1.100800e+05
   2.201600e+05 -1.100800e+05 -2.201600e+05 -1.088000e+04 -2.176000e+04
   1.088000e+04  2.176000e+04]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  3.413408e+06
   4.198656e+06  0.000000e+00  0.000000e+00 -9.929600e+04 -1.985920e+05
   9.929600e+04  1.985920e+05]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   8.926144e+06  0.000000e+00  0.000000

In [4]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):-110719.99983616, ('q2','q2'):-201343.99967232, ('q3','q3'):130815.99983616, ('q4','q4'):281727.99967232, ('q5','q5'):3413408.00008192, ('q6','q6'):8926144.00016384, ('q7','q7'):-1314080.00008192, ('q8','q8'):-528832.0001638401, ('q9','q9'):-118179.99997952, ('q10','q10'):-231231.99995904, ('q11','q11'):123307.99997952, ('q12','q12'):251743.99995904}

quadratic = {('q1','q2'):40192.0, ('q1','q5'):-55040.0, ('q1','q6'):-110080.0, ('q1','q7'):55040.0, ('q1','q8'):110080.0, ('q1','q9'):5440.0, ('q1','q10'):10880.0, ('q1','q11'):-5440.0, ('q1','q12'):-10880.0, ('q2','q5'):-110080.0, ('q2','q6'):-220160.0, ('q2','q7'):110080.0, ('q2','q8'):220160.0, ('q2','q9'):10880.0, ('q2','q10'):21760.0, ('q2','q11'):-10880.0, ('q2','q12'):-21760.0, ('q3','q4'):40192.0, ('q3','q5'):55040.0, ('q3','q6'):110080.0, ('q3','q7'):-55040.0, ('q3','q8'):-110080.0, ('q3','q9'):-5440.0, ('q3','q10'):-10880.0, ('q3','q11'):5440.0, ('q3','q12'):10880.0, ('q4','q5'):110080.0, ('q4','q6'):220160.0, ('q4','q7'):-110080.0, ('q4','q8'):-220160.0, ('q4','q9'):-10880.0, ('q4','q10'):-21760.0, ('q4','q11'):10880.0, ('q4','q12'):21760.0, ('q5','q6'):4198656.0, ('q5','q9'):-99296.0, ('q5','q10'):-198592.0, ('q5','q11'):99296.0, ('q5','q12'):198592.0, ('q6','q9'):-198592.0, ('q6','q10'):-397184.0, ('q6','q11'):198592.0, ('q6','q12'):397184.0, ('q7','q8'):4198656.0, ('q7','q9'):99296.0, ('q7','q10'):198592.0, ('q7','q11'):-99296.0, ('q7','q12'):-198592.0, ('q8','q9'):198592.0, ('q8','q10'):397184.0, ('q8','q11'):-198592.0, ('q8','q12'):-397184.0, ('q9','q10'):10256.0, ('q11','q12'):10256.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9          energy num_oc. chain_b.
0    1   0   0   0  1  0  0  0  0  1  0  1  -1423395.99957       7      0.0
1    1   0   0   0  1  0  0  0  0  1  0  0  -1420831.99959       9      0.0
2    1   1   0   0  1  0  0  0  0  1  0  0 -1420831.999549       9      0.0
3    0   1   0   0  1  0  0  0  0  1  0  0 -1416223.999713      10      0.0
4    1   0   1   0  1  0  0  0  0  1  0  1  -1415703.99959       3      0.0
5    0   1   0   0  1  0  0  0  0  1  0  1 -1413971.999693      17      0.0
6    0   0   0   0  1  0  0  0  0  1  0  1 -1413347.999734      10      0.0
7    1   0   1   0  1  0  0  0  0  1  0  0 -1413139.999611       3      0.0
8    1   1   1   0  1  0  0  0  0  1  0  0  -1413139.99957       3      0.0
9    1   1   0   0  1  0  0  0  0  1  0  1 -1413139.999529      18      0.0
10   1   1   1   0  1  0  0  0  0  1  0  1 -1405447.999549       7      0.0
11   0   0   0   0  1  0  0  0  0  1  0  0 -1405343.999754       9      0.0
12   0   1  

In [5]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9          energy num_oc. chain_b.
0    1   0   0   0  1  0  0  0  0  1  0  1  -1423395.99957      10      0.0
1    1   0   0   0  1  0  0  0  0  1  0  0  -1420831.99959       1      0.0
2    1   1   0   0  1  0  0  0  0  1  0  0 -1420831.999549       8      0.0
3    0   1   0   0  1  0  0  0  0  1  0  0 -1416223.999713      14      0.0
4    1   0   1   0  1  0  0  0  0  1  0  1  -1415703.99959       2      0.0
5    0   1   0   0  1  0  0  0  0  1  0  1 -1413971.999693      10      0.0
6    0   0   0   0  1  0  0  0  0  1  0  1 -1413347.999734       5      0.0
7    1   0   1   0  1  0  0  0  0  1  0  0 -1413139.999611       1      0.0
8    1   1   1   0  1  0  0  0  0  1  0  0  -1413139.99957       1      0.0
9    1   1   0   0  1  0  0  0  0  1  0  1 -1413139.999529      10      0.0
270  1   1   0   0  1  0  0  0  0  1  0  1 -1413139.999529       1 0.083333
10   1   1   1   0  1  0  0  0  0  1  0  1 -1405447.999549       2      0.0
11   0   0  

In [6]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9          energy num_oc. chain_b.
0    1   0   0   0  1  0  0  0  0  1  0  1  -1423395.99957      10      0.0
1    1   0   0   0  1  0  0  0  0  1  0  0  -1420831.99959      11      0.0
2    1   1   0   0  1  0  0  0  0  1  0  0 -1420831.999549      16      0.0
3    0   1   0   0  1  0  0  0  0  1  0  0 -1416223.999713      11      0.0
4    1   0   1   0  1  0  0  0  0  1  0  1  -1415703.99959       5      0.0
5    0   1   0   0  1  0  0  0  0  1  0  1 -1413971.999693      12      0.0
6    0   0   0   0  1  0  0  0  0  1  0  1 -1413347.999734      13      0.0
7    1   0   1   0  1  0  0  0  0  1  0  0 -1413139.999611       9      0.0
8    1   1   1   0  1  0  0  0  0  1  0  0  -1413139.99957      10      0.0
9    1   1   0   0  1  0  0  0  0  1  0  1 -1413139.999529       7      0.0
10   1   1   1   0  1  0  0  0  0  1  0  1 -1405447.999549      14      0.0
11   0   0   0   0  1  0  0  0  0  1  0  0 -1405343.999754      16      0.0
12   0   1  

In [7]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9          energy num_oc. chain_b.
0    1   0   0   0  1  0  0  0  0  1  0  1  -1423395.99957       4      0.0
1    1   1   0   0  1  0  0  0  0  1  0  0 -1420831.999549       4      0.0
2    0   1   0   0  1  0  0  0  0  1  0  0 -1416223.999713       4      0.0
3    1   0   1   0  1  0  0  0  0  1  0  1  -1415703.99959       4      0.0
4    0   1   0   0  1  0  0  0  0  1  0  1 -1413971.999693       1      0.0
5    0   0   0   0  1  0  0  0  0  1  0  1 -1413347.999734       4      0.0
211  0   0   0   0  1  0  0  0  0  1  0  1 -1413347.999734       1 0.083333
6    1   0   1   0  1  0  0  0  0  1  0  0 -1413139.999611       6      0.0
7    1   1   1   0  1  0  0  0  0  1  0  0  -1413139.99957       4      0.0
8    1   1   0   0  1  0  0  0  0  1  0  1 -1413139.999529       2      0.0
9    1   1   1   0  1  0  0  0  0  1  0  1 -1405447.999549       3      0.0
10   0   0   0   0  1  0  0  0  0  1  0  0 -1405343.999754      14      0.0
11   0   1  

In [8]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9          energy num_oc. chain_b.
0    1   0   0   0  1  0  0  0  0  1  0  1  -1423395.99957       7      0.0
1    1   0   0   0  1  0  0  0  0  1  0  0  -1420831.99959      10      0.0
2    1   1   0   0  1  0  0  0  0  1  0  0 -1420831.999549      11      0.0
3    0   1   0   0  1  0  0  0  0  1  0  0 -1416223.999713       7      0.0
4    1   0   1   0  1  0  0  0  0  1  0  1  -1415703.99959      11      0.0
5    0   1   0   0  1  0  0  0  0  1  0  1 -1413971.999693       5      0.0
6    0   0   0   0  1  0  0  0  0  1  0  1 -1413347.999734       3      0.0
7    1   0   1   0  1  0  0  0  0  1  0  0 -1413139.999611      16      0.0
8    1   1   1   0  1  0  0  0  0  1  0  0  -1413139.99957      13      0.0
9    1   1   0   0  1  0  0  0  0  1  0  1 -1413139.999529       3      0.0
10   1   1   1   0  1  0  0  0  0  1  0  1 -1405447.999549      14      0.0
11   0   0   0   0  1  0  0  0  0  1  0  0 -1405343.999754       4      0.0
12   0   1  

In [9]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=10000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9          energy num_oc. chain_b.
0    1   0   0   0  1  0  0  0  0  1  0  1  -1423395.99957      73      0.0
1    1   0   0   0  1  0  0  0  0  1  0  0  -1420831.99959      29      0.0
438  1   0   0   0  1  0  0  0  0  1  0  0  -1420831.99959       1 0.083333
2    1   1   0   0  1  0  0  0  0  1  0  0 -1420831.999549      23      0.0
3    0   1   0   0  1  0  0  0  0  1  0  0 -1416223.999713      41      0.0
4    1   0   1   0  1  0  0  0  0  1  0  1  -1415703.99959      70      0.0
5    0   1   0   0  1  0  0  0  0  1  0  1 -1413971.999693      89      0.0
6    0   0   0   0  1  0  0  0  0  1  0  1 -1413347.999734     131      0.0
7    1   0   1   0  1  0  0  0  0  1  0  0 -1413139.999611      31      0.0
8    1   1   1   0  1  0  0  0  0  1  0  0  -1413139.99957      20      0.0
9    1   1   0   0  1  0  0  0  0  1  0  1 -1413139.999529      34      0.0
457  1   1   0   0  1  0  0  0  0  1  0  1 -1413139.999529       1 0.083333
10   1   1  

In [10]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=10000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9          energy num_oc. chain_b.
0    1   0   0   0  1  0  0  0  0  1  0  1  -1423395.99957      58      0.0
1    1   0   0   0  1  0  0  0  0  1  0  0  -1420831.99959      63      0.0
498  1   0   0   0  1  0  0  0  0  1  0  0  -1420831.99959       1 0.083333
2    1   1   0   0  1  0  0  0  0  1  0  0 -1420831.999549      86      0.0
3    0   1   0   0  1  0  0  0  0  1  0  0 -1416223.999713     207      0.0
4    1   0   1   0  1  0  0  0  0  1  0  1  -1415703.99959      31      0.0
437  1   0   1   0  1  0  0  0  0  1  0  1  -1415703.99959       1 0.083333
5    0   1   0   0  1  0  0  0  0  1  0  1 -1413971.999693     156      0.0
439  0   1   0   0  1  0  0  0  0  1  0  1 -1413971.999693       1 0.083333
511  0   1   0   0  1  0  0  0  0  1  0  1 -1413971.999693       1 0.083333
6    0   0   0   0  1  0  0  0  0  1  0  1 -1413347.999734     131      0.0
7    1   0   1   0  1  0  0  0  0  1  0  0 -1413139.999611      53      0.0
8    1   1  

In [11]:
sampleset = sampler_auto.sample_qubo(Q, num_reads=10000)
print(sampleset)

    q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9          energy num_oc. chain_b.
0    1   0   0   0  1  0  0  0  0  1  0  1  -1423395.99957      46      0.0
1    1   0   0   0  1  0  0  0  0  1  0  0  -1420831.99959      41      0.0
2    1   1   0   0  1  0  0  0  0  1  0  0 -1420831.999549      50      0.0
3    0   1   0   0  1  0  0  0  0  1  0  0 -1416223.999713      68      0.0
4    1   0   1   0  1  0  0  0  0  1  0  1  -1415703.99959      31      0.0
5    0   1   0   0  1  0  0  0  0  1  0  1 -1413971.999693      77      0.0
6    0   0   0   0  1  0  0  0  0  1  0  1 -1413347.999734      54      0.0
7    1   0   1   0  1  0  0  0  0  1  0  0 -1413139.999611      29      0.0
8    1   1   1   0  1  0  0  0  0  1  0  0  -1413139.99957      39      0.0
9    1   1   0   0  1  0  0  0  0  1  0  1 -1413139.999529      55      0.0
10   1   1   1   0  1  0  0  0  0  1  0  1 -1405447.999549      37      0.0
11   0   0   0   0  1  0  0  0  0  1  0  0 -1405343.999754      56      0.0
12   0   1  